In [2]:
%store -r __importData
import pandas as pd
import numpy as np
import ggplot as gplt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [3]:
__importData

import group_table


C:\Users\S\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


import airline_table


C:\Users\S\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


import order_table
import train_set
import test_set


In [6]:
def vacation_feature(x):
    nearest_weekend = np.argsort(np.abs(x["begin_date"] - long_weekend["long_weekend_date"]))[0]
    nearest_weekend = long_weekend.iloc[nearest_weekend]["long_weekend_id"]
    return nearest_weekend

long_weekend = pd.DataFrame(columns=["long_weekend_id", "long_weekend_date", "long_weekend_days",
                                     "strategy_weekend_days", "cost_days"], index=range(16))
long_weekend["long_weekend_date"] = ["2016-12-31", "2017-01-27", "2017-02-25", "2017-04-01", "2017-04-29",
                                     "2017-05-27", "2017-10-04", "2017-10-07", "2017-12-30", "2018-02-15",
                                     "2018-02-28", "2018-04-04", "2018-04-28","2018-06-16", "2018-09-22", "2018-10-10"]
long_weekend["long_weekend_days"] = [3, 6, 4, 4, 3, 6, 1, 4,
                                     3, 6, 1, 5, 3, 3, 3, 1]
long_weekend["strategy_weekend_days"] = [3, 10, 9, 9, 9, 9, 7, 4,
                                         3, 11, 5, 9, 4, 3, 3, 5]
long_weekend["cost_days"] = [0, 2, 3, 3, 4, 3, 2, 0,
                             0, 3, 2, 2, 1, 0, 0, 2]
long_weekend["long_weekend_date"] = pd.to_datetime(long_weekend["long_weekend_date"])
long_weekend["long_weekend_id"] = list(range(16))

In [7]:
group_table["begin_date"] = pd.to_datetime(group_table["begin_date"])
s = time.clock()
group_table["long_weekend_id"] = group_table.apply(vacation_feature, axis=1)
e = time.clock()
print(e-s)

80.95490218574207


In [9]:
group_table = pd.merge(group_table, long_weekend, on="long_weekend_id", how="left")

In [13]:
group_table[["group_id"] + list(group_table.columns[-4:])].to_csv("long_weekend_feature.csv", encoding="utf-8", index=False)

In [16]:
group_table = group_table[["group_id", "days", "begin_date"] + list(group_table.columns[-4:])]

In [17]:
def vacation_extra_feat(input_table):
    table = input_table
    table["long_weekend_dist"] = list(map(lambda x: x.days, table["begin_date"] - table["long_weekend_date"]))
    table["days_weekenddays_diff"] = table["days"] - table["long_weekend_days"]
    table["days_strategy_diff"] = table["days"] - table["strategy_weekend_days"]
    return table

In [18]:
final_vac_feat = vacation_extra_feat(group_table)

C:\Users\S\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\S\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\S\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [20]:
final_vac_feat.drop(columns=["begin_date", "days", "long_weekend_date"]).to_csv("long_weekend_feature_2.csv",
                                                                                encoding="utf-8", index=False)